# Cap'n Freako's Solutions to the Exercises in *Arrows and Computation*, by Ross Paterson

Original author: David Banas <capn.freako@gmail.com>  
Original date:   July 29, 2016

Copyright &#169; 2016 David Banas; all rights reserved World wide.


In [1]:
import Data.Time

putStrLn "Notebook last run:"
getCurrentTime


Notebook last run:

2016-08-10 14:19:08.350943 UTC

## Table of Contents <a name="contents"/>

* <a href="#ex1">Exercise 1 - *Arrow* instances for *Reader* and *Writer*.</a>
* <a href="#ex2">Exercise 2 - Attempted *Arrow* instance for *ListMap*.</a>


## Exercise 1 <a name="ex1"/>

Write *Arrow* instances for the following types:

`newtype Reader s a b = R ((s, a)` &#8594; `b)`  
`newtype Writer a b   = W (a` &#8594; `(String, b))`

In the latter case, any monoid could be used in place of String.

In [2]:
import Prelude hiding (pure)

class Arrow a where
    pure  :: (b -> c) -> a b c
    (>>>) :: a b c -> a c d -> a b d
    first :: a b c -> a (b, d) (c, d)

second :: Arrow a => a b c -> a (d, b) (d, c)
second f = pure swap >>> first f >>> pure swap
    where swap ~(x, y) = (y, x)

assoc :: ((a, b), c) -> (a, (b, c))
assoc ~(~(x, y), z) = (x, (y, z))

unassoc :: (a, (b, c)) -> ((a, b), c)
unassoc ~(x, ~(y, z)) = ((x, y), z)

prod :: (a -> a') -> (b -> b') -> (a, b) -> (a', b')
(f `prod` g) ~(x, y) = (f x, g y)

newtype Reader s a b = R {runR :: (s, a) -> b}

newtype Writer a b   = W {runW :: a -> (String, b)}

instance Arrow (Reader s) where
    pure f      = R (\(_, x) -> f x)
    R f >>> R g = R (\(s, x) -> g (s, f (s, x)))
    first (R f) = R (\(s, (x, y)) -> (f (s, x), y))
    
instance Arrow Writer where
    pure f      = W (\x -> ("", f x))
    W f >>> W g = W (\x -> 
        let (s', w) = f x
            (s, y)  = g w
        in (s' ++ s, y))
    first (W f) = W (\(x, y) -> 
        let (s, w) = f x
        in (s, (w, y)))

-- Test axioms.
a1 = R (\(s, x) -> s)
a2 = R (\(s, x) -> x)
a3 = R (\(s, x) -> ())
b1 = W (\x -> ("", x))
b2 = W (\x -> (show x, x + 1))
b3 = W (\x -> ("Hello, World!", x * 2))
x  = ("test", 1)
y  = ("for_first", (2, 3))
z  = ("for_assoc", ((2, 3), 4))
f  = \x -> x + 1
g  = \x -> x * 2
w  = (2, 3)
v = ((2, 3), 4)

-- General utilities
eqR :: (Eq b) => Reader s a b -> Reader s a b -> (s, a) -> Bool
eqR r1 r2 x = runR r1 x == runR r2 x

eqW :: (Eq b) => Writer a b -> Writer a b -> a -> Bool
eqW w1 w2 x = runW w1 x == runW w2 x

-- Functor identity
assertion1 = eqR a1 a' x && eqR a1 a'' x
    where a'  = idA >>> a1
          a'' = a1 >>> idA
          idA = pure id

assertion1w = eqW b1 a' 1 && eqW b1 a'' 1
    where a'  = idA >>> b1
          a'' = b1 >>> idA
          idA = pure id

-- Functor composition
assertion2 = eqR (pure (g . f)) (pure f >>> pure g) x
assertion2w = eqW (pure (g . f)) (pure f >>> pure g) 1

-- Associativity
assertion3 = eqR ((a1 >>> a2) >>> a3) (a1 >>> (a2 >>> a3)) x
assertion3w = eqW ((b1 >>> b2) >>> b3) (b1 >>> (b2 >>> b3)) 1

-- Extension
assertion4 = eqR (first (pure f)) (pure (f `prod` id)) y
assertion4w = eqW (first (pure f)) (pure (f `prod` id)) w

-- Functor
assertion5 = eqR (first (a1 >>> a2)) (first a1 >>> first a2) y
assertion5w = eqW (first (b1 >>> b2)) (first b1 >>> first b2) w

-- Exchange
assertion6 = eqR (first a1 >>> pure (id `prod` f)) (pure (id `prod` f) >>> first a1) y
assertion6w = eqW (first b1 >>> pure (id `prod` f)) (pure (id `prod` f) >>> first b1) w

-- Unit
assertion7 = eqR (first a1 >>> pure fst) (pure fst >>> a1) y
assertion7w = eqW (first b1 >>> pure fst) (pure fst >>> b1) w

-- Association
assertion8 = eqR (first (first a1) >>> pure assoc) (pure assoc >>> first a1) z
assertion8w = eqW (first (first b1) >>> pure assoc) (pure assoc >>> first b1) v

main :: IO ()
main = do
    print assertion1
    print assertion1w
    print assertion2
    print assertion2w
    print assertion3
    print assertion3w
    print assertion4
    print assertion4w
    print assertion5
    print assertion5w
    print assertion6
    print assertion6w
    print assertion7
    print assertion7w
    print assertion8
    print assertion8w

main


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True

<a href="#contents">Back to Contents</a>

## Exercise 2 <a name="ex2"/>

The following is almost an arrow type:

`newtype ListMap ι o = LM ([ι] → [o])`

What goes wrong?

In [3]:
newtype ListMap a b = LM {runLM :: [a] -> [b]}

instance Arrow ListMap where
    pure f        = LM (\xs -> map f xs)
    LM f >>> LM g = LM (\xs -> (g . f) xs)
    first (LM f)  = LM (\ws ->  -- ws :: [(a, b)]
        let (xs, ys) = unzip ws
        in zip (f xs) ys)

-- Test axioms.
-- General utilities
eqLM :: (Eq b) => ListMap a b -> ListMap a b -> [a] -> Bool
eqLM lm1 lm2 xs = runLM lm1 xs == runLM lm2 xs

-- Useful expressions
lm1 = LM (\xs -> map (+1) xs)
lm2 = LM (\xs -> map (*2) xs)
lm3 = LM (\xs -> map (id) xs)
xs  = [0, 1, 2]
ys  = [(0, "zero"), (1, "one"), (2, "two")]
zs  = [(0, 1), (1, 2), (2, 3)]
zss = [((0, 1), 2), ((1, 2), 3), ((2, 3), 4)]

-- Functor identity
assertion_lm1 = eqLM lm1 a' xs && eqLM lm1 a'' xs
    where a'  = idA >>> lm1
          a'' = lm1 >>> idA
          idA = pure id

-- Functor composition
assertion_lm2 = eqLM (pure (g . f)) (pure f >>> pure g) xs

-- Associativity
assertion_lm3 = eqLM ((lm1 >>> lm2) >>> lm3) (lm1 >>> (lm2 >>> lm3)) xs

-- Extension
assertion_lm4 = eqLM (first (pure f)) (pure (f `prod` id)) ys

-- Functor
assertion_lm5 = eqLM (first (lm1 >>> lm2)) (first lm1 >>> first lm2) ys

-- Exchange
assertion_lm6 = eqLM (first lm1 >>> pure (id `prod` f)) (pure (id `prod` f) >>> first lm1) zs

-- Unit
assertion_lm7 = eqLM (first lm1 >>> pure fst) (pure fst >>> lm1) ys

-- Association
assertion_lm8 = eqLM (first (first lm1) >>> pure assoc) (pure assoc >>> first lm1) zss

main :: IO ()
main = do
    print assertion_lm1
    print assertion_lm2
    print assertion_lm3
    print assertion_lm4
    print assertion_lm5
    print assertion_lm6
    print assertion_lm7
    print assertion_lm8

main


True
True
True
True
True
True
True
True

<a href="#contents">Back to Contents</a>